In [77]:
import pandas as pd
import numpy as np

#### the data set contains 14000 building footprint from DADAR region of mumbai and hazards are rainfall(mm) and wind speed (m/s) from NASA POWER of mumbai and outer region of last 1 year

In [78]:
ws = pd.read_csv('wind_speed.csv')
rf = pd.read_csv('rainfall.csv')

In [79]:
ws.head()

,LAT,LON,YEAR,MO,DY,WS10M
0,18.5,71.875,2025,2,1,6.74
1,18.5,72.500,2025,2,1,6.53
2,18.5,73.125,2025,2,1,3.20
3,18.5,73.750,2025,2,1,2.62
4,18.5,74.375,2025,2,1,2.77


In [80]:
ws['WS10M'] = ws['WS10M'].mean()

In [81]:
ws.head()

,LAT,LON,YEAR,MO,DY,WS10M
0,18.5,71.875,2025,2,1,4.5261
1,18.5,72.500,2025,2,1,4.5261
2,18.5,73.125,2025,2,1,4.5261
3,18.5,73.750,2025,2,1,4.5261
4,18.5,74.375,2025,2,1,4.5261


In [82]:
rf.head()

,LAT,LON,YEAR,MO,DY,PRECTOTCORR
0,18.5,71.875,2025,2,1,0.0
1,18.5,72.500,2025,2,1,0.0
2,18.5,73.125,2025,2,1,0.0
3,18.5,73.750,2025,2,1,0.0
4,18.5,74.375,2025,2,1,0.0


In [83]:
rf['PRECTOTCORR'] = rf['PRECTOTCORR'].mean()

In [84]:
rf.head()

,LAT,LON,YEAR,MO,DY,PRECTOTCORR
0,18.5,71.875,2025,2,1,5.496347
1,18.5,72.500,2025,2,1,5.496347
2,18.5,73.125,2025,2,1,5.496347
3,18.5,73.750,2025,2,1,5.496347
4,18.5,74.375,2025,2,1,5.496347


In [87]:
import pandas as pd

# Clean column names (important)
rf.columns = rf.columns.str.strip().str.upper()
ws.columns = ws.columns.str.strip().str.upper()

# Convert to integer (safe step)
for df in [rf, ws]:
    df["YEAR"] = df["YEAR"].astype(int)
    df["MO"] = df["MO"].astype(int)
    df["DY"] = df["DY"].astype(int)

    # Create DATE column
    df["DATE"] = pd.to_datetime(
        dict(year=df["YEAR"], month=df["MO"], day=df["DY"])
    )

    # Drop old columns
    df.drop(columns=["YEAR","MO","DY"], inplace=True)

print("Rainfall columns:", rf.columns)
print("Wind columns:", ws.columns)

Rainfall columns: Index(['LAT', 'LON', 'PRECTOTCORR', 'DATE'], dtype='object')
Wind columns: Index(['LAT', 'LON', 'WS10M', 'DATE'], dtype='object')


In [88]:
hazard_df = pd.merge(
    rf,
    ws,
    on=["LAT","LON","DATE"],
    how="inner"
)

In [89]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# ---------------------------------
# 1️⃣ Load Buildings
# ---------------------------------
buildings = gpd.read_file("building_map.shp")

# Ensure lat/lon CRS
buildings = buildings.to_crs("EPSG:4326")

# Create building_id
buildings["building_id"] = range(1, len(buildings)+1)

# Get centroid coordinates
buildings["latitude"] = buildings.geometry.centroid.y
buildings["longitude"] = buildings.geometry.centroid.x


# ---------------------------------
# 2️⃣ Prepare Hazard Grid Points
# ---------------------------------
# hazard_df should already have:
# LAT | LON | precipitation | wind_10m

hazard_grid = hazard_df.groupby(["LAT","LON"]).mean().reset_index()

hazard_coords = hazard_grid[["LAT","LON"]].values

# Build KDTree
tree = cKDTree(hazard_coords)


# ---------------------------------
# 3️⃣ Map Each Building to Nearest Grid
# ---------------------------------
building_coords = buildings[["latitude","longitude"]].values

distances, indices = tree.query(building_coords)

# Assign hazard values
buildings["PRECTOTCORR"] = hazard_grid.iloc[indices]["PRECTOTCORR"].values
buildings["WS10M"] = hazard_grid.iloc[indices]["WS10M"].values


print("✅ Nearest hazard assigned")
print(buildings[["building_id","PRECTOTCORR","WS10M"]].head())

C:\Users\user\AppData\Local\Temp\ipykernel_10200\3598622985.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings["latitude"] = buildings.geometry.centroid.y
C:\Users\user\AppData\Local\Temp\ipykernel_10200\3598622985.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings["longitude"] = buildings.geometry.centroid.x


✅ Nearest hazard assigned
   building_id  PRECTOTCORR   WS10M
0            1     5.496347  4.5261
1            2     5.496347  4.5261
2            3     5.496347  4.5261
3            4     5.496347  4.5261
4            5     5.496347  4.5261


In [90]:
hazard_df.head()

,LAT,LON,PRECTOTCORR,DATE,WS10M
0,18.5,71.875,5.496347,2025-02-01,4.5261
1,18.5,72.500,5.496347,2025-02-01,4.5261
2,18.5,73.125,5.496347,2025-02-01,4.5261
3,18.5,73.750,5.496347,2025-02-01,4.5261
4,18.5,74.375,5.496347,2025-02-01,4.5261


In [91]:
hazard_df.shape

(7300, 5)

#### here finding closest hazard lat long to each building lat long and assigningthat hazard value to each closest building

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree


# Load Buildings

buildings = gpd.read_file("building_map.shp")
buildings = buildings.to_crs("EPSG:4326")
buildings["building_id"] = range(1, len(buildings)+1)
buildings["latitude"] = buildings.geometry.centroid.y
buildings["longitude"] = buildings.geometry.centroid.x


#  Prepare Hazard Grid Points

# hazard_df already has individual hazard records:
# LAT | LON | PRECTOTCORR | WS10M
hazard_coords = hazard_df[["LAT","LON"]].values

# Build KDTree for nearest neighbor search
tree = cKDTree(hazard_coords)

# ---------------------------------
# Map Each Building to Nearest Hazard Point
# ---------------------------------
building_coords = buildings[["latitude","longitude"]].values
distances, indices = tree.query(building_coords)

# Assign hazard values from original hazard_df (no mean)
buildings["PRECTOTCORR"] = hazard_df.iloc[indices]["PRECTOTCORR"].values
buildings["WS10M"] = hazard_df.iloc[indices]["WS10M"].values

print("Nearest hazard assigned without averaging")
print(buildings[["building_id","PRECTOTCORR","WS10M"]].head())

C:\Users\user\AppData\Local\Temp\ipykernel_10200\3103700342.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings["latitude"] = buildings.geometry.centroid.y
C:\Users\user\AppData\Local\Temp\ipykernel_10200\3103700342.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings["longitude"] = buildings.geometry.centroid.x


✅ Nearest hazard assigned without averaging
   building_id  PRECTOTCORR   WS10M
0            1     5.496347  4.5261
1            2     5.496347  4.5261
2            3     5.496347  4.5261
3            4     5.496347  4.5261
4            5     5.496347  4.5261


In [93]:
print(buildings["PRECTOTCORR"].describe())
print(buildings["WS10M"].describe())

count    1.441500e+04
mean     5.496347e+00
std      1.990477e-12
min      5.496347e+00
25%      5.496347e+00
50%      5.496347e+00
75%      5.496347e+00
max      5.496347e+00
Name: PRECTOTCORR, dtype: float64
count    1.441500e+04
mean     4.526100e+00
std      1.057857e-12
min      4.526100e+00
25%      4.526100e+00
50%      4.526100e+00
75%      4.526100e+00
max      4.526100e+00
Name: WS10M, dtype: float64


#### Prectotcorr(rainfall) and ws10m is same because building dataset is of very small so each building has been assigned same value since paramter doesnt change for very small region


In [94]:
buildings.head()

,latitude,longitude,area_in_me,confidence,full_plus_,geometry,building_id,PRECTOTCORR,WS10M
0,18.993732,72.841604,207.9815,0.6849,7JCJXRVR+FJX7,"POLYGON ((72.84159 18.99362, 72.84153 18.99363...",1,5.496347,4.5261
1,18.981326,72.840379,31.4906,0.6506,7JCJXRJR+G5J2,"POLYGON ((72.8404 18.9813, 72.84036 18.98129, ...",2,5.496347,4.5261
2,18.992025,72.818691,44.8961,0.7536,7JCJXRR9+RF82,"POLYGON ((72.81874 18.99201, 72.81866 18.99199...",3,5.496347,4.5261
3,18.995464,72.846418,44.1015,0.7991,7JCJXRWW+5HMF,"POLYGON ((72.84646 18.99549, 72.84643 18.99542...",4,5.496347,4.5261
4,18.999262,72.817490,392.0109,0.8931,7JCJXRX8+PX5G,"POLYGON ((72.81761 18.99932, 72.81753 18.99911...",5,5.496347,4.5261


In [95]:
buildings.columns = buildings.columns.str.strip()

In [99]:
buildings.drop(['confidence','full_plus_','geometry'],axis=1,inplace=True)

In [97]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Correct column selection
buildings[['PRECTOTCORR','WS10M']] = scaler.fit_transform(buildings[['PRECTOTCORR','WS10M']])

print(buildings[['PRECTOTCORR','WS10M']].head())

    PRECTOTCORR  WS10M
0  8.881784e-16    0.0
1  8.881784e-16    0.0
2  8.881784e-16    0.0
3  8.881784e-16    0.0
4  8.881784e-16    0.0


In [100]:
buildings.head()

,latitude,longitude,area_in_me,building_id,PRECTOTCORR,WS10M
0,18.993732,72.841604,207.9815,1,8.881784e-16,0.0
1,18.981326,72.840379,31.4906,2,8.881784e-16,0.0
2,18.992025,72.818691,44.8961,3,8.881784e-16,0.0
3,18.995464,72.846418,44.1015,4,8.881784e-16,0.0
4,18.999262,72.817490,392.0109,5,8.881784e-16,0.0


In [101]:
buildings['exposure_score'] = buildings['PRECTOTCORR'] + buildings['WS10M']

In [102]:
buildings.head()

,latitude,longitude,area_in_me,building_id,PRECTOTCORR,WS10M,exposure_score
0,18.993732,72.841604,207.9815,1,8.881784e-16,0.0,8.881784e-16
1,18.981326,72.840379,31.4906,2,8.881784e-16,0.0,8.881784e-16
2,18.992025,72.818691,44.8961,3,8.881784e-16,0.0,8.881784e-16
3,18.995464,72.846418,44.1015,4,8.881784e-16,0.0,8.881784e-16
4,18.999262,72.817490,392.0109,5,8.881784e-16,0.0,8.881784e-16


In [103]:
buildings.to_csv("buildings_exposure.csv", index=False)